In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression


In [2]:
data = pd.read_csv('tested.csv')
data

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,0,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,1,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,0,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,0,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,1,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S
...,...,...,...,...,...,...,...,...,...,...,...,...
413,1305,0,3,"Spector, Mr. Woolf",male,NaN,0,0,A.5. 3236,8.0500,NaN,S
414,1306,1,1,"Oliva y Ocana, Dona. Fermina",female,39.0,0,0,PC 17758,108.9000,C105,C
415,1307,0,3,"Saether, Mr. Simon Sivertsen",male,38.5,0,0,SOTON/O.Q. 3101262,7.2500,NaN,S
416,1308,0,3,"Ware, Mr. Frederick",male,NaN,0,0,359309,8.0500,NaN,S


In [3]:
data = data.set_index('PassengerId')
data.head()

,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
PassengerId,,,,,,,,,,,
892,0,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
893,1,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
894,0,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
895,0,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
896,1,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S


In [4]:
target = 'Survived'

In [5]:
y = data[target]


In [6]:
x = data.drop(target, axis=1)

#### Изучение качаства данных и очистка

In [7]:
x.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 418 entries, 892 to 1309
Data columns (total 10 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Pclass    418 non-null    int64  
 1   Name      418 non-null    object 
 2   Sex       418 non-null    object 
 3   Age       332 non-null    float64
 4   SibSp     418 non-null    int64  
 5   Parch     418 non-null    int64  
 6   Ticket    418 non-null    object 
 7   Fare      417 non-null    float64
 8   Cabin     91 non-null     object 
 9   Embarked  418 non-null    object 
dtypes: float64(2), int64(3), object(5)
memory usage: 35.9+ KB


В поле Cabin много нанов, поля Name и Ticket тоже использовать не будем

In [8]:
x = x.drop(['Cabin', 'Name', 'Ticket'], axis=1)

в поле Age 20% значений не заполненно, заменим их на средние

In [9]:
mean_age = x['Age'].mean()
mean_age

30.272590361445783

In [10]:
x['Age'] = x['Age'].fillna(mean_age)

In [11]:
x['Age'].unique()

array([34.5       , 47.        , 62.        , 27.        , 22.        ,
       14.        , 30.        , 26.        , 18.        , 21.        ,
       30.27259036, 46.        , 23.        , 63.        , 24.        ,
       35.        , 45.        , 55.        ,  9.        , 48.        ,
       50.        , 22.5       , 41.        , 33.        , 18.5       ,
       25.        , 39.        , 60.        , 36.        , 20.        ,
       28.        , 10.        , 17.        , 32.        , 13.        ,
       31.        , 29.        , 28.5       , 32.5       ,  6.        ,
       67.        , 49.        ,  2.        , 76.        , 43.        ,
       16.        ,  1.        , 12.        , 42.        , 53.        ,
       26.5       , 40.        , 61.        , 60.5       ,  7.        ,
       15.        , 54.        , 64.        , 37.        , 34.        ,
       11.5       ,  8.        ,  0.33      , 38.        , 57.        ,
       40.5       ,  0.92      , 19.        , 36.5       ,  0.75

#### Работа с категориальными признаками

In [12]:
x['Sex'].value_counts()

male      266
female    152
Name: Sex, dtype: int64

In [13]:
x['Sex'] = (x['Sex'] == 'female').astype(int)

In [14]:
x['Sex'].value_counts()

0    266
1    152
Name: Sex, dtype: int64

In [15]:
x.head()

,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
PassengerId,,,,,,,
892,3,0,34.5,0,0,7.8292,Q
893,3,1,47.0,1,0,7.0000,S
894,2,0,62.0,0,0,9.6875,Q
895,3,0,27.0,0,0,8.6625,S
896,3,1,22.0,1,1,12.2875,S


In [16]:
x = pd.get_dummies(x)
x.head()

,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked_C,Embarked_Q,Embarked_S
PassengerId,,,,,,,,,
892,3,0,34.5,0,0,7.8292,0,1,0
893,3,1,47.0,1,0,7.0000,0,0,1
894,2,0,62.0,0,0,9.6875,0,1,0
895,3,0,27.0,0,0,8.6625,0,0,1
896,3,1,22.0,1,1,12.2875,0,0,1


In [17]:
x['Pclass'] = x['Pclass'].astype('category')
x.head()

,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked_C,Embarked_Q,Embarked_S
PassengerId,,,,,,,,,
892,3,0,34.5,0,0,7.8292,0,1,0
893,3,1,47.0,1,0,7.0000,0,0,1
894,2,0,62.0,0,0,9.6875,0,1,0
895,3,0,27.0,0,0,8.6625,0,0,1
896,3,1,22.0,1,1,12.2875,0,0,1


In [18]:
x['Pclass'].unique()

[3, 2, 1]
Categories (3, int64): [1, 2, 3]

In [19]:
x = pd.get_dummies(x)
x.head()

,Sex,Age,SibSp,Parch,Fare,Embarked_C,Embarked_Q,Embarked_S,Pclass_1,Pclass_2,Pclass_3
PassengerId,,,,,,,,,,,
892,0,34.5,0,0,7.8292,0,1,0,0,0,1
893,1,47.0,1,0,7.0000,0,0,1,0,0,1
894,0,62.0,0,0,9.6875,0,1,0,0,1,0
895,0,27.0,0,0,8.6625,0,0,1,0,0,1
896,1,22.0,1,1,12.2875,0,0,1,0,0,1


In [20]:
x_train, x_valid, y_train, y_valid = train_test_split(x, y, test_size=0.25, random_state=42)

In [21]:
x_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 313 entries, 1024 to 994
Data columns (total 11 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Sex         313 non-null    int64  
 1   Age         313 non-null    float64
 2   SibSp       313 non-null    int64  
 3   Parch       313 non-null    int64  
 4   Fare        312 non-null    float64
 5   Embarked_C  313 non-null    uint8  
 6   Embarked_Q  313 non-null    uint8  
 7   Embarked_S  313 non-null    uint8  
 8   Pclass_1    313 non-null    uint8  
 9   Pclass_2    313 non-null    uint8  
 10  Pclass_3    313 non-null    uint8  
dtypes: float64(2), int64(3), uint8(6)
memory usage: 16.5 KB


In [22]:
lr = LogisticRegression()

In [23]:
lr.fit(x_train, y_train)

ValueError: Input X contains NaN.
LogisticRegression does not accept missing values encoded as NaN natively. For supervised learning, you might want to consider sklearn.ensemble.HistGradientBoostingClassifier and Regressor which accept missing values encoded as NaNs natively. Alternatively, it is possible to preprocess the data, for instance by using an imputer transformer in a pipeline or drop samples with missing values. See https://scikit-learn.org/stable/modules/impute.html You can find a list of all estimators that handle NaN values at the following page: https://scikit-learn.org/stable/modules/impute.html#estimators-that-handle-nan-values

In [24]:
x.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 418 entries, 892 to 1309
Data columns (total 11 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Sex         418 non-null    int64  
 1   Age         418 non-null    float64
 2   SibSp       418 non-null    int64  
 3   Parch       418 non-null    int64  
 4   Fare        417 non-null    float64
 5   Embarked_C  418 non-null    uint8  
 6   Embarked_Q  418 non-null    uint8  
 7   Embarked_S  418 non-null    uint8  
 8   Pclass_1    418 non-null    uint8  
 9   Pclass_2    418 non-null    uint8  
 10  Pclass_3    418 non-null    uint8  
dtypes: float64(2), int64(3), uint8(6)
memory usage: 22.0 KB


In [25]:
mean_fare = x['Fare'].mean()
mean_fare

35.627188489208635

In [26]:
x['Fare'] = x['Fare'].fillna(mean_fare)
x.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 418 entries, 892 to 1309
Data columns (total 11 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Sex         418 non-null    int64  
 1   Age         418 non-null    float64
 2   SibSp       418 non-null    int64  
 3   Parch       418 non-null    int64  
 4   Fare        418 non-null    float64
 5   Embarked_C  418 non-null    uint8  
 6   Embarked_Q  418 non-null    uint8  
 7   Embarked_S  418 non-null    uint8  
 8   Pclass_1    418 non-null    uint8  
 9   Pclass_2    418 non-null    uint8  
 10  Pclass_3    418 non-null    uint8  
dtypes: float64(2), int64(3), uint8(6)
memory usage: 22.0 KB


In [27]:
x_train, x_valid, y_train, y_valid = train_test_split(x, y, test_size=0.25, random_state=42)

In [28]:
lr.fit(x_train, y_train)

/home/sk/jupyter_projects/gb_Python_DS/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression()

In [29]:
y_pred = lr.predict(x_valid)
y_pred

array([0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0,
       1, 0, 1, 1, 1, 0, 0, 0, 0, 1, 1, 0, 1, 0, 1, 0, 1, 0, 1, 1, 0, 0,
       0, 0, 1, 1, 1, 0, 0, 1, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0,
       0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1])

In [30]:
from sklearn.metrics import accuracy_score

#### Метрика Accuracty - доля правильных ответов

In [32]:
accuracy_score(y_valid, y_pred)


1.0

### Вычисление верочтности событий

In [33]:
y_proba = lr.predict_proba(x_valid)

Вероятности событий 0 и 1 для каждого пассажира, нас интересует вероятность события 1 (правый столбец)

In [35]:
y_proba

array([[0.97221633, 0.02778367],
       [0.03831832, 0.96168168],
       [0.95946273, 0.04053727],
       [0.97841114, 0.02158886],
       [0.06714396, 0.93285604],
       [0.97420603, 0.02579397],
       [0.07541772, 0.92458228],
       [0.97471056, 0.02528944],
       [0.05818289, 0.94181711],
       [0.97771381, 0.02228619],
       [0.97337354, 0.02662646],
       [0.06109278, 0.93890722],
       [0.05186134, 0.94813866],
       [0.04213026, 0.95786974],
       [0.97771381, 0.02228619],
       [0.97450355, 0.02549645],
       [0.07497573, 0.92502427],
       [0.97402123, 0.02597877],
       [0.94273641, 0.05726359],
       [0.97047244, 0.02952756],
       [0.97708527, 0.02291473],
       [0.97031725, 0.02968275],
       [0.05644588, 0.94355412],
       [0.9726062 , 0.0273938 ],
       [0.05357206, 0.94642794],
       [0.04185651, 0.95814349],
       [0.04295829, 0.95704171],
       [0.97133981, 0.02866019],
       [0.9703908 , 0.0296092 ],
       [0.9726062 , 0.0273938 ],
       [0.

In [37]:
x_train.to_pickle('x_train.pkl')
y_train.to_pickle('y_train.pkl')

x_valid.to_pickle('x_valid.pkl')
y_valid.to_pickle('y_valid.pkl')

In [38]:
ls -l


итого 332
-rw-rw-r-- 1 sk sk  34742 янв 21 13:43  boston_house_prices.csv
-rw-rw-r-- 1 sk sk 154198 янв 24 21:44 'Less_3.sklearn_Линейная  регрессия.ipynb'
-rw-rw-r-- 1 sk sk  69719 янв 26 17:11 'Less_3.sklearn_Логистическая  регрессия.ipynb'
-rw-rw-r-- 1 sk sk  29474 янв 24 22:25  tested.csv
-rw-rw-r-- 1 sk sk  17908 янв 26 17:12  x_train.pkl
-rw-rw-r-- 1 sk sk   6672 янв 26 17:12  x_valid.pkl
-rw-rw-r-- 1 sk sk   8141 янв 26 17:12  y_train.pkl
-rw-rw-r-- 1 sk sk   3145 янв 26 17:12  y_valid.pkl
